<font size=5>**使用python和MySQL的接口查询数据，并使用plotly进行数据的可视化**</font>

所使用的数据来自于MySQL的经典数据库world，包含country/countrylanguage/city共三张表格

本文将结合使用SQL和pandas包进行基础查询

In [7]:
import pandas as pd
import numpy as np
import pymysql
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
conn = pymysql.connect(host="localhost", user="root", password="********", database="world" )
cursor = conn.cursor()

通过SQL查询世界上各种语言的使用总人数，并按降序排列

In [69]:
sql = '''
SELECT Language, round(sum(country.Population*countryLanguage.Percentage)/1000000, 2) as Population_millions
FROM country INNER JOIN countrylanguage
On country.Code = countryLanguage.CountryCode
GROUP BY Language
ORDER BY Population_millions DESC
'''
cursor.execute(sql)
df = pd.DataFrame(np.array(cursor.fetchall()))
df.rename(columns={0:'Language', 1:'Population_millions'}, inplace=True)
df.iloc[:15]

,Language,Population_millions
0,Chinese,119184.35
1,Hindi,40563.31
2,Spanish,35502.95
3,English,34707.79
4,Arabic,23383.92
5,Bengali,20930.47
6,Portuguese,17759.53
7,Russian,16080.76
8,Japanese,12681.41
9,Punjabi,10402.54


查询各种语言被认定为国家官方用语的国家数量

In [72]:
sql = '''
SELECT Language, count(*) as Countries
FROM countryLanguage
WHERE IsOfficial = 'T'
GROUP BY Language
ORDER BY Countries DESC
'''
cursor.execute(sql)
df = pd.DataFrame(np.array(cursor.fetchall()))
df.rename(columns={0:'Language', 1:'Countries'}, inplace=True)
df.iloc[:15]

,Language,Countries
0,English,44
1,Arabic,22
2,Spanish,20
3,French,18
4,German,6
5,Portuguese,6
6,Italian,4
7,Dutch,4
8,Malay,4
9,Russian,3


查询世界各国的首位城市及其对应的人口数量

In [77]:
sql = '''
SELECT CountryCode, country.Name, city.Name, city.Population
FROM city INNER JOIN country
ON city.CountryCode = country.Code
GROUP BY CountryCode
ORDER BY Population DESC
'''
cursor.execute(sql)
df = pd.DataFrame(np.array(cursor.fetchall()))
df.rename(columns={0:'Code', 1:'Country', 2:'City', 3:'Population'}, inplace=True)
df.iloc[:15]

,Code,Country,City,Population
0,IND,India,Mumbai (Bombay),10500000
1,KOR,South Korea,Seoul,9981619
2,BRA,Brazil,São Paulo,9968485
3,CHN,China,Shanghai,9696300
4,IDN,Indonesia,Jakarta,9604900
5,PAK,Pakistan,Karachi,9269265
6,TUR,Turkey,Istanbul,8787958
7,MEX,Mexico,Ciudad de México,8591309
8,RUS,Russian Federation,Moscow,8389200
9,USA,United States,New York,8008278


查询各国的国名、所属大洲、人口、预期寿命、国民生产总值等基本信息

In [26]:
sql = '''
SELECT Name, Continent, Population, LifeExpectancy, GNP
FROM country
'''
cursor.execute(sql)
df = pd.DataFrame(np.array(cursor.fetchall()))
df.rename(columns={0:'Name', 1:'Continent', 2:'Population', 3:'LifeExpectancy', 4:'GNP'}, inplace=True)
df.sort_values('LifeExpectancy').iloc[:10]

,Name,Continent,Population,LifeExpectancy,GNP
237,Zambia,Africa,9169000,37.2,3377
143,Mozambique,Africa,19680000,37.5,2891
148,Malawi,Africa,10925000,37.6,1687
238,Zimbabwe,Africa,11669000,37.8,5951
2,Angola,Africa,12878000,38.3,6648
35,Botswana,Africa,1622000,39.3,4834
182,Rwanda,Africa,7733000,39.3,2036
201,Swaziland,Africa,1008000,40.4,1206
153,Niger,Africa,10730000,41.3,1706
151,Namibia,Africa,1726000,42.5,3101


将每个国家的国民生产总值和出生时预期寿命通过散点图展示出来

In [28]:
trace = go.Scatter(
    x=df['GNP'],
    y=df['LifeExpectancy'],
    text= df['Name'] ,
    mode='markers'
)

layout = go.Layout(
    title='Life expectancy vs GNP from MySQL world database',
    xaxis=go.layout.XAxis( type='log', title='GNP' ),
    yaxis=go.layout.YAxis( title='Life expectancy' ),
)

data = [trace]
fig = go.Figure(data, layout=layout)
py.iplot(fig, filename='world GNP vs life expectancy')

对上面的散点图进行美化修饰，通过气泡图更好地展现结果

In [60]:
# 对于不同的大洲绘制不同的颜色
colors = {
    'Asia':"rgb(255,65,54)", 
    'Europe':"rgb(133,20,75)",
    'Africa':"rgb(0,116,217)",
    'North America':"rgb(255,133,27)",
    'South America':"rgb(23,190,207)",
    'Antarctica':"rgb(61,153,112)",
    'Oceania':"rgb(255,220,0)",
}

# 设置参照系，控制每个气泡的面积大小
sizeref = df['Population'].max()/5000

# 对图像上每个点的信息进行展示
def make_text(X):
    return 'Country: %s\
            <br>Life Expectancy: %s years\
            <br>Population: %s millions'\
            %(X['Name'], X['LifeExpectancy'], X['Population']/1e6)
    

# 生成trace
def make_trace(X, continent, sizes, color):
    return go.Scatter(
                x=X['GNP'],
                y=X['LifeExpectancy'],
                name=continent,
                text=X.apply(make_text, axis=1).tolist(),
                mode='markers',
                marker=go.scatter.Marker(
                                    color=color,
                                    size=sizes,
                                    sizemode='area',
                                    sizeref=sizeref
                                        )
                )



In [61]:
# 对于各大洲的数据生成trace
data = list()
for continent, X in df.groupby('Continent'):
    
    sizes = X['Population']
    color = colors[continent]
    data.append(make_trace(X, continent, sizes, color))

# 设置图像的layout
layout = go.Layout(
    title="Life expectancy vs GNP from MySQL world database (bubble chart)",
    plot_bgcolor='#EFECEA',
    xaxis=go.layout.XAxis(
                    type='log',
                    title="Gross National Product",
                    range=[2,7.5],
                    #ticklen=8,
                    #tickwidth=1.5
                    ),
    yaxis=go.layout.YAxis(
                    type='log',
                    title="Life Expectancy [in years]",
                    #ticklen=8,
                    #tickwidth=1.5
                    )
)

# 绘制气泡图
fig = go.Figure(data, layout)
py.iplot(fig, filename='world GNP vs life expectancy')